In [1]:
#loading the data

import pandas as pd

bank_df = pd.read_csv('bankdata.csv.crdownload')

In [2]:
#split the data in fraud and not fraud cases

result_0 = bank_df.loc[bank_df['fraud'] == 0]
used_data0=result_0.loc[:49]


result_1 = bank_df.loc[bank_df['fraud'] == 1]
result_1 = result_1.reset_index()
used_data1=result_1.loc[:49]


#combined the frames and reset the index
frames = [used_data0, used_data1] 

result = pd.concat(frames)
result = result.reset_index()



#cleaning the data

result['age'] = result['age'].str.replace('[^0-9]', '', regex=True).astype(int) #(.astype(int))note that this is buggy for more than 466 data points
result['zipcodeOri'] = result['zipcodeOri'].str.replace('[^0-9]', '', regex=True)
result['zipMerchant'] = result['zipMerchant'].str.replace('[^0-9]', '', regex=True)
result['merchant'] = result['merchant'].str[2:-1].astype(int)
result['customer'] = result['customer'].str[2:-1].astype(int)


from sklearn.preprocessing import LabelEncoder
ce = LabelEncoder()
result['category'] = ce.fit_transform(result['category'])
result['gender'] = ce.fit_transform(result['gender'])


#dropping features determined to be insignificant 

result=result.loc[:, result.columns.drop(['customer', 'zipcodeOri', 'merchant', 'zipMerchant','step','index', 'level_0'])]

labels = result['fraud'] #y_labels


#data examining 
result.dtypes
result.values
labels.values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [3]:
#include this line if running on qbraid
# !qbraid jobs enable default

In [ ]:
%%time

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from qiskit import Aer
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit_machine_learning.kernels import QuantumKernel

from qiskit_ibm_provider import IBMProvider

# provider = IBMProvider(instance='ibm-q-startup/qbraid/main')
provider = IBMProvider(instance='ibm-q-education/codershq-1/nyuad-2023')
backend = provider.get_backend('ibmq_qasm_simulator')


# Load the credit card fraud detection dataset
# df = pd.read_csv('creditcard.csv')
# Split the dataset into training and testing sets
X = result.iloc[:, :-1]
y = result.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Encode the training and testing data onto a quantum circuit using the ZZFeatureMap
feature_map = ZZFeatureMap(X.shape[1], reps=2)
# backend = Aer.get_backend('qasm_simulator')
print('*'*100)
qkernel = QuantumKernel(feature_map,quantum_instance=backend)
# X_train_encoded = qkernel.evaluate(X_train)
# X_test_encoded = qkernel.evaluate(X_test)
# Set up the QSVC algorithm
print('check 1: instantiating qvc')
svm = QSVC(quantum_kernel=qkernel        
        )
# Train the QSVC algorithm on the training data
print('check 2: training')
svm.fit(X_train, y_train)
print('check 3: done')
# Test the QSVC algorithm on the testing data
y_pred = svm.predict(X_test)
print('check 4: done predict')
# Print the accuracy of the QSVC algorithm
Accuracy = np.mean(y_pred == y_test)
print("Accuracy:", Accuracy)


for i in range(1000000):
    pass

****************************************************************************************************
check 1: instantiating qvc
check 2: training
